Lower-bounding f_n and h_n using a mixed integer programming framework (with the Branch-and-cut algorithm).

In [1]:
%run constants.ipynb
%run gen_h.ipynb
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat
import re

In [2]:
def var_name(*ineqs, prefix=''):
    '''
    Returns the variable name for the MIP.
    '''
    return prefix + f'{str(ineqs)}'

In [3]:
def extract_ineq(var):
    '''
    Extracts the inequality from the given variable's name.
    (For interpretability, we associate each indicator variable's name 
    with the inequality(s) they represent)
    '''
    match = re.findall(r'\(([^()]+)\)', var.varName)
    return sympify(match[0]) if match else None

In [4]:
def extract_ineqs(variables):
    '''
    Extracts inequalities from multiple variables, 
    based on their values (i.e. 1 = true, 0 = false).
    '''
    ineqs = []
    for v in variables:
        e = extract_ineq(v)
        if v.x == 1 and e is not None:  # indicator is true
            ineqs.append(e)
        elif v.x == 0:  # indicator is false
            ineqs.append(Not(And(e)))
    return ineqs

In [5]:
def to_gp_linexpr(sp_expr, var_map):
    '''
    Converts a sympy expression to a gurobipy linear expression,
    using the provided (sp : gp) variable mapping.
    '''
    gp_expr = gp.LinExpr()

    if isinstance(sp_expr, Add):
        for term in sp_expr.args:
            gp_expr.add(to_gp_linexpr(term, var_map))
    elif sp_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sp_expr.args:
            if factor.is_number:  # sp coefficient
                coeff *= float(factor)
            elif factor in var_map:  # sp variable
                var = var_map[factor]
        if var is not None:
            gp_expr.addTerms(coeff, var)
        else:  # constant
            gp_expr.addConstant(coeff)
    elif sp_expr in var_map:  # single variable expression
        gp_expr.addTerms(1.0, var_map[sp_expr])
    elif sp_expr.is_number:   # single constant
        gp_expr.addConstant(float(sp_expr))

    return gp_expr

In [6]:
EPS = 1e-6
def unify_ineq(ineq, eps, scale_by=1):
    '''
    Converts the given inequality to the form expr >= 0.
    '''
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(scale_by * (lhs - rhs), 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= eps
        return Rel(scale_by * (lhs - rhs) - eps, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(scale_by * (rhs - lhs), 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= eps
        return Rel(scale_by * (rhs - lhs - eps), 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [7]:
def to_gp_ineq(ineq, var_map, scale_by=1, eps=EPS):
    '''
    Like `to_gp_linexpr`, but converts a sympy inequality to a gurobipy inequality 
    (of the form >= 0).
    '''
    return (to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map) >= 0)

In [8]:
def add_ind_constr(ineq, model=None, name='ind', var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds indicator constraint to the model for a single inequality. 
    '''
    lhs = to_gp_linexpr(unify_ineq(ineq, eps, scale_by=scale_by).lhs, var_map)  # to the form >= 0
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -eps))  # lhs < 0
    return v

In [9]:
def add_ind_constrs(*ineqs, model=None, var_map=None, eps=EPS, scale_by=1):
    ''' 
    Adds variables and constraints to the model for a compound indicator 
    (i.e. may represent the and of multiple inequalities). 
    '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], 
                               name=var_name(ineqs[0], prefix='single_ind'), 
                               var_map=var_map,
                               model=model,
                               scale_by=scale_by,
                               eps=eps),)
    
    I = model.addVar(vtype=GRB.BINARY, 
                     name=var_name(*ineqs, prefix='main_ind'))
    component_vars = [add_ind_constr(i, 
                                     name=var_name(i, prefix='component_ind'), 
                                     var_map=var_map,
                                     model=model,
                                     scale_by=scale_by,
                                     eps=eps) for i in ineqs]
    model.addConstr(I == gp.and_(component_vars), 
                    name=var_name(*ineqs, prefix='andconstr'))
    return I, component_vars

In [10]:
def analyse_result(model, relax=False):
    '''
    Analyses the status of the model.
    If the model is feasible and `relax` is True, then re-optimizes the model with relaxed bounds.
    Note that if the relaxation is infeasible, then the exact model must too be infeasible
    '''
    if model.status == GRB.OPTIMAL:
        print('Optimal solution found:')
        for v in model.getVars():
            print(f"{v.varName}: {v.x}")
        print(f'Objective value:', model.objVal)
        print(f'MIPGap = |ObjBound - ObjVal|/|ObjVal|:', model.MIPGap)
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the constraints and try to make the model feasible
        print('The model is infeasible')
        if relax:
            print('Relaxing the bounds')
            model.feasRelaxS(1, False, False, True)
            model.optimize()
            analyse_result(model, relax=relax)

In [26]:
EPS=1

In [32]:
def f_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Runs MIP model to minimize f_n - thresh(n).
    If `output_file` is True, writes the model as a '.lp' file.
    If `analyse` is True, analyses the optimization results.
    '''
    # uncomment the following to collect the regions cut out (i.e. excluded) by the model
#     failed_regions = []
    
#     def extract_region(model):
#         region = []
#         variables = model.getVars()
#         sol = model.cbGetSolution(variables)
#         for v, val in zip(variables, sol):
#             # only look at the individual indicators
#             if not 'single_ind' in v.varName and not 'component_ind' in v.varName:
#                 continue
#             e = extract_ineq(v)
#             if val == 1 and e is not None:
#                 region.append(e[0])
#             elif val == 0:
#                 # indicator is false
#                 # guaranteed single inequality
#                 region.append(Not(e[0]))
#         return region
    
#     def pos_f_callback(model, where):
#         if where == GRB.Callback.MIPSOL:
#             # Capture node information
#             try:
#                 obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
#                 print("Current objective:", obj)
#                 if obj > 0:  # Node is infeasible
#                     failed_regions.append(extract_region(model))
#             except gp.GurobiError as e:
#                 print(f"Gurobi error during callback: {e}")
#             except Exception as e:
#                 print(f"Unexpected error during callback: {e}")
    
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    num_pos_per_n = [len(h_xyz[i][1]) for i in range(1,n+1)]
    num_neg_per_n = [len(h_yxz[i][1]) for i in range(1,n+1)]
    num_pos = sum(num_pos_per_n)
    num_neg = sum(num_neg_per_n)
    
    pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
    neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
    const = sum([coef(i)*(h_xyz[i][0] - h_yxz[i][0]) for i in range(1,n+1)])
    
    all_pos_comp_inds = concat([h_xyz[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([h_yxz[i][1] for i in range(1,n+1)])
    
    model = gp.Model(F_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
#     model.Params.MIPFocus = 1
#     model.Params.FeasibilityTol = 1e-9
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_neg_comp_inds]

#     pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
#                 for i in all_pos_comp_inds]
#     neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp) \
#                 for i in all_neg_comp_inds]
    
    obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
#     # add extra constraints
#     for ind in pos_inds:
#         for p in ind[2]:
#             model.addConstr()
    
    
    # constraints for V
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')

#     for i, ineq in enumerate(V):
#         v = add_ind_constr(ineq, model=model, var_map=sp_to_gp, name='V: ' + str(ineq))
#         model.addConstr(v == 1)
        
    f_constr = model.addConstr(obj <= 0, name='non_pos_f')

#     model.addConstr(gx >= 1, name=f'V0')
#     model.addConstr(gy >= gx + 1, name=f'V1')
#     model.addConstr(gz >= gy + 1, name=f'V2')
    
#     model.optimize(pos_f_callback)
    model.setObjective(obj, GRB.MINIMIZE)
#     model.setObjective(0, GRB.MAXIMIZE)
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(F_MIP_OUTPUT(n))
    
    return ((model.status == GRB.OPTIMAL and \
            model.MIPGap == 0 and \
            sub(V, {x:gx.x, y:gy.x, z:gz.x}) == 1 and \
            eval_f(n, gx.x, gy.x, gz.x) < 0), # sanity check using the x,y,z values from the model
            model)

In [34]:
f_mip(3, analyse=True, relax=False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 111 columns and 71 nonzeros
Model fingerprint: 0xa493feee
Model has 195 general constraints
Variable types: 3 continuous, 108 integer (108 binary)
Coefficient statistics:
  Matrix range     [5e-03, 1e+00]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-02, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 7e+00]
Presolve added 291 rows and 261 columns
Presolve time: 0.01s
Presolved: 295 rows, 372 columns, 907 nonzeros
Presolved model has 174 SOS constraint(s)
Variable types: 177 continuous, 195 integer (195 binary)

Root relaxation: objective -1.648148e-01, 158 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    C

(True,
 <gurobi.Model MIP instance f3(x,y,z)-alpha3: 4 constrs, 111 vars, Parameter changes: Username=(user-defined)>)

In [29]:
eval_f(3,11,12,17)

-0.016666666666666677

# LB on h_n(x,y,z) - h_n(y,x,z)

In [35]:
def h_mip(n, output_file=False, analyse=True, relax=False):
    '''
    Like `f_mip`, but runs MIP model to minimize h_n(x,y,z) - h_n(y,x,z).
    '''
    h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
    h_yxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
    
    all_pos_comp_inds = concat([h_xyz[i][1] for i in range(1,n+1)])
    all_neg_comp_inds = concat([h_yxz[i][1] for i in range(1,n+1)])
    
    model = gp.Model(DH_MODEL_NAME(n), env=gp.Env())
    model.Params.OutputFlag = 1
    model.Params.LogToConsole = 1
    model.Params.MIPFocus = 1
    
    gx = model.addVar(lb=0, name='x')
    gy = model.addVar(lb=0, name='y')
    gz = model.addVar(lb=0, name='z')
    sp_to_gp = {x: gx, y: gy, z: gz}
    
    pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in h_xyz[n][1]]
    neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) for i in h_yxz[n][1]]
    const = h_xyz[n][0] - h_yxz[n][0]
    
    obj = coef(n) * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
    
    model.setObjective(obj, GRB.MINIMIZE)
    for i, ineq in enumerate(V):
        model.addConstr(to_gp_ineq(ineq, sp_to_gp, eps=EPS), name=f'V{i}')
    
    model.optimize()
    
    if analyse:
        analyse_result(model, relax=relax)
    
    if output_file:
        model.write(DH_MIP_OUTPUT(n))
    
    return (model.status == GRB.OPTIMAL and model.MIPGap == 0), model

In [37]:
h_mip(4, analyse=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 799 columns and 5 nonzeros
Model fingerprint: 0x59493f94
Model has 1370 general constraints
Variable types: 3 continuous, 796 integer (796 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2072 rows and 1722 columns
Presolve time: 0.02s
Presolved: 2075 rows, 2521 columns, 6037 nonzeros
Presolved model has 1148 SOS constraint(s)
Variable types: 1151 continuous, 1370 integer (1370 binary)

Root relaxation: objective -1.118827e-01, 1172 iterations, 0.00 se

(True,
 <gurobi.Model MIP instance h4(x,y,z)-h4(y,x,z): 3 constrs, 799 vars, Parameter changes: MIPFocus=1, Username=(user-defined)>)

In [38]:
eval_dh(4,22,23,68)

Fraction(-11, 648)

# f(x,x,z) - f(x+1,x-1,z)

In [12]:
n=5
h_xyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
h_xxz = read(H_XXZ_CACHE+'_xyz', min_n=1, max_n=n)

In [13]:
num_pos_per_n = [len(h_xyz[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(h_xxz[i][1]) for i in range(1,n+1)]
num_pos = sum(num_pos_per_n)
num_neg = sum(num_neg_per_n)

pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
const = sum([coef(i)*(h_xyz[i][0] - h_xxz[i][0]) for i in range(1,n+1)])

/var/folders/tk/2zq7clf94gx3rdh5slf6rhc00000gn/T/ipykernel_1082/3349906455.py:14: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not isinstance(args, collections.Hashable):


In [14]:
all_pos_comp_inds = concat([h_xyz[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([h_xxz[i][1] for i in range(1,n+1)])

## Real version

In [48]:
EPS = 1
SCALE=10000

In [49]:
model = gp.Model(f"dh'{n}_model", env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1

gx = model.addVar(lb=EPS, ub=SCALE, name='x')
gy = model.addVar(lb=EPS, ub=SCALE, vtype='I', name='y')
gz = model.addVar(lb=Z_LB * SCALE, vtype='I', ub=SCALE, name='z')
# dummy variable for the x in h(x,x,z), since this x'=(1-z)/2=(x+y)/2
sp_to_gp = {x: gx, y: gy, z: gz}

# constraints for V
model.addConstr(gx + gy + gz == SCALE, name=f'x+y+z=1')
model.addConstr(gx <= gy - EPS, name=f'x<y')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1


<gurobi.Constr *Awaiting Model Update*>

In [50]:
pos_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
                for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i, model=model, var_map=sp_to_gp, eps=EPS) \
            for i in all_neg_comp_inds]

In [51]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)
    
model.setObjective(obj, GRB.MINIMIZE)

In [52]:
f_constr = model.addConstr(obj <= 0, name='non_pos_f')

In [53]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5856 columns and 1895 nonzeros
Model fingerprint: 0x9a011c5a
Model has 10156 general constraints
Variable types: 0 continuous, 5856 integer (5853 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [1e-04, 3e-02]
  Bounds range     [1e+00, 1e+04]
  RHS range        [3e-02, 1e+04]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 3e+01]
Presolve added 10440 rows and 0 columns
Presolve removed 0 rows and 87 columns
Presolve time: 0.48s
Presolved: 10443 rows, 5769 columns, 39265 nonzeros
Variable types: 0 continuous, 5769 integer (5766 binary)

Root relaxation: objective -1.976494e-01, 6407 iterations, 0.59 seconds (1.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [42]:
analyse_result(model)

Optimal solution found:
x: 308.0
y: 346.0
z: 346.0
single_ind(3*x <= y,): 0.0
single_ind(2*x <= z,): 0.0
single_ind(2*x <= y,): 0.0
single_ind(3*x <= z,): -0.0
single_ind(y < z,): 0.0
single_ind(x <= -y + z,): 0.0
single_ind(x <= y - z,): -0.0
single_ind(z < y,): 0.0
single_ind(7*x <= y,): -0.0
main_ind(4*x <= z, 3*x < y): -0.0
component_ind(4*x <= z,): 0.0
component_ind(3*x < y,): 0.0
single_ind(5*x <= y,): -0.0
single_ind(6*x <= z,): -0.0
main_ind(5*x <= 3*y, y < 3*x): 0.0
component_ind(5*x <= 3*y,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(3*x <= y + z, y < 3*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(y < 3*x,): 1.0
main_ind(2*x <= y, y - z < x): 0.0
component_ind(2*x <= y,): 0.0
component_ind(y - z < x,): 1.0
single_ind(x <= -y + z,): -0.0
main_ind(3*x <= y + z, z < 2*x): 0.0
component_ind(3*x <= y + z,): 0.0
component_ind(z < 2*x,): 1.0
single_ind(z < 2*x,): 1.0
single_ind(3*x <= y - z,): 0.0
single_ind(x < y - z,): -0.0
single_ind(6*x <= y,): 0.0
main_ind(5*x <= z, 

In [54]:
%run constants.ipynb
%run gen_h.ipynb
%run verification.ipynb

In [46]:
eval_d_dash(n, model.getVarByName('x').x *10, model.getVarByName('y').x *10, model.getVarByName('z').x *10)

-0.05830761316872428

In [55]:
delta_dash(n, model.getVarByName('x').x *10, model.getVarByName('y').x *10, model.getVarByName('z').x *10)

-0.05830761316872428

In [340]:
delta_dash(7, 332,334,334)

-0.03958261888431641

In [341]:
delta_dash(8,332,334,334)

-0.036458214258497185

In [ ]:
for i in range(9,12):
    print(delta_dash(i,332,334,334))

-0.03489680577782858
-0.03409205503586005
-0.033695550715824994


In [ ]:
delta_dash(12,332,334,334)